In [12]:
import numpy as np 
import cv2 
import glob

# 加载之前保存的参数
with np.load('left.npz') as X:
    mtx, dist, _, __ = [X[i] for i in ('mtx', 'dist', 'rvecs', 'tvecs')]
    # mtx, dist, rvecs, tvecs = [X[i] for i in ('mtx', 'dist', 'rvecs', 'tvecs')]


# 画 3D 坐标轴
def draw_line(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    ## 注意，此处的数据一定要转化成 int32 格式，否则 OpenCV 报错
    img = cv2.line(img, np.int32(corner), np.int32(tuple(imgpts[0].ravel())), (255, 0, 0), 5)
    img = cv2.line(img, np.int32(corner), np.int32(tuple(imgpts[1].ravel())), (0, 255, 0), 5)
    img = cv2.line(img, np.int32(corner), np.int32(tuple(imgpts[2].ravel())), (0, 0, 255), 5)
    return img

# 画 3D 立方体
def draw_cube(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    print(imgpts)
    # 绿色画棋盘面
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    
    # 蓝色画柱子
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
        
    # 红色画顶
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img


criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

axis = np.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]).reshape(-1, 3)

axis_cube = np.float32([[0, 0, 0], [0, 3, 0], [3, 3, 0], [3, 0, 0],
                            [0, 0, -3], [0, 3, -3], [3, 3, -3], [3, 0, -3]])

for fname in glob.glob('assets/left/left*.jpg'):
    img = cv2.imread(fname)
    img_cube = cv2.imread(fname)
    
    print(fname)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)
    if ret == True:
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        
        # 找到旋转和平移向量
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, dist)
        
        # 将3D点映射到平面上
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw_line(img, corners2, imgpts)
        
        # cube point
        imgpts_cube, jac_cube = cv2.projectPoints(axis_cube, rvecs, tvecs, mtx, dist)        
        img_cube = draw_cube(img_cube, corners, imgpts_cube)
        
        cv2.imshow('img', img)        
        cv2.imshow('img_cube', img_cube)
        k = cv2.waitKey(0) & 0xFF
        if k == ord('s'):
            cv2.imwrite(fname[:-4]+'_result.jpg', img)
cv2.destroyAllWindows()


assets/left\left01.jpg
[[475 264]
 [477 158]
 [372 157]
 [372 259]
 [471 292]
 [474 161]
 [344 160]
 [345 285]]
assets/left\left02.jpg
[[254 308]
 [375 335]
 [396 242]
 [251 212]
 [210 412]
 [351 447]
 [372 363]
 [194 321]]
assets/left\left03.jpg
[[497 374]
 [541 235]
 [406 190]
 [355 323]
 [530 364]
 [581 177]
 [402 121]
 [333 294]]
assets/left\left04.jpg
[[475 338]
 [473 197]
 [339 201]
 [337 335]
 [476 353]
 [474 164]
 [296 171]
 [293 348]]
assets/left\left05.jpg
[[279 378]
 [439 341]
 [399 202]
 [258 237]
 [224 333]
 [446 281]
 [391 107]
 [207 154]]
assets/left\left06.jpg
[[393 357]
 [489 376]
 [503 275]
 [404 260]
 [353 385]
 [466 411]
 [484 289]
 [364 270]]
assets/left\left07.jpg
[[159 306]
 [234 340]
 [265 246]
 [188 219]
 [ 98 302]
 [178 343]
 [215 231]
 [130 200]]
assets/left\left08.jpg
[[196 328]
 [320 359]
 [355 227]
 [235 208]
 [124 291]
 [275 330]
 [324 162]
 [179 145]]
assets/left\left09.jpg
assets/left\left11.jpg
assets/left\left12.jpg
[[200 360]
 [346 360]
 [345 220]
 [